In [1]:
import pandas as pd
from pathlib import Path

from config import driver, username, password, host, port, database

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func


In [2]:
QuestionnaireDF = pd.read_csv(Path('../resources/data.csv'), delimiter='\t')
QuestionnaireDF.rename(columns ={'country':'COUNTRY', 
                                 'dateload':'DATELOAD',
                                 'introelapse':'INTROELAPSE',
                                 'testelapse':'TESTELAPSE',
                                 'surveyelapse':'SURVEYELAPSE',
                                 'gender':'GENDER',
                                 'engnat':'ENGNAT',
                                 'age':'AGE',
                                 'IE':'INTROVERT_EXTROVERT'}, inplace=True)
QuestionnaireDF.head()

,Q1A,Q1I,Q1E,Q2A,Q2I,Q2E,Q3A,Q3I,Q3E,Q4A,...,Q91E,COUNTRY,DATELOAD,INTROELAPSE,TESTELAPSE,SURVEYELAPSE,GENDER,ENGNAT,AGE,INTROVERT_EXTROVERT
0,5,51,7107,3,91,2522,1,56,6180,2,...,4609,US,2019-02-20 17:35:52,1,461,16,2,1,23,3
1,5,39,6354,5,13,3092,1,12,5243,5,...,10409,AU,2019-02-20 17:46:32,21,467,15,1,2,25,2
2,3,17,5397,4,35,2747,5,40,5262,3,...,2691,BR,2019-02-20 18:10:24,56,306,17,1,2,19,1
3,5,41,3055,2,14,3348,1,13,5141,1,...,3697,CZ,2019-02-20 18:16:21,2,287,14,1,1,23,1
4,1,76,2542,2,54,1878,1,15,5637,1,...,1662,CA,2019-02-20 18:21:49,2,325,12,1,1,18,2


In [3]:
QuestionsListDF = pd.read_csv(Path('../resources/QuestionsList.csv'))
QuestionsListDF.head()

,QuestionNum,Question
0,Q1,"""I would never audition to be on a game show."""
1,Q2,"""I am not much of a flirt."""
2,Q3,"""I have to psych myself up before I am brave e..."
3,Q4,"""I would hate living with room mates."""
4,Q5,"""I mostly listen to people in conversations."""


In [4]:
connection_string = f"{driver}://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)
connection = engine.connect()

In [5]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

#View list of tables
Base.classes.keys()

session = Session(engine)

['questionslist', 'questionnaire']

In [6]:
# Save reference to the tables
questionsListTB = Base.classes.questionslist
questionnaireTB = Base.classes.questionnaire

In [14]:
session.query(questionsListTB).delete()
QuestionsListDF.to_sql('questionslist',connection, if_exists='append', index=False)

0

In [7]:
session.query(questionnaireTB).delete()
QuestionnaireDF.to_sql('questionnaire',connection, if_exists='append', index=False)

0

In [13]:
QuestionnaireSurveyDF = pd.read_sql_table('questionnaire', connection)
QuestionnaireSurveyDF

,row_id,Q1A,Q1I,Q1E,Q2A,Q2I,Q2E,Q3A,Q3I,Q3E,...,Q91E,COUNTRY,DATELOAD,INTROELAPSE,TESTELAPSE,SURVEYELAPSE,GENDER,ENGNAT,AGE,INTROVERT_EXTROVERT
0,1,5,51,7107,3,91,2522,1,56,6180,...,4609,US,2019-02-20 17:35:52,1,461,16,2,1,23,3
1,2,5,39,6354,5,13,3092,1,12,5243,...,10409,AU,2019-02-20 17:46:32,21,467,15,1,2,25,2
2,3,3,17,5397,4,35,2747,5,40,5262,...,2691,BR,2019-02-20 18:10:24,56,306,17,1,2,19,1
3,4,5,41,3055,2,14,3348,1,13,5141,...,3697,CZ,2019-02-20 18:16:21,2,287,14,1,1,23,1
4,5,1,76,2542,2,54,1878,1,15,5637,...,1662,CA,2019-02-20 18:21:49,2,325,12,1,1,18,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7183,7184,1,46,1328,4,82,3214,4,43,3360,...,3495,US,2019-08-19 12:54:22,8,299,14,2,1,53,1
7184,7185,2,5,8786,5,24,2233,5,10,10387,...,6088,CA,2019-08-19 13:10:30,25,498,20,1,1,20,1
7185,7186,3,29,6618,5,44,2393,4,58,5768,...,3425,BY,2019-08-19 13:29:47,3,326,17,2,2,28,1
7186,7187,4,15,8321,2,18,6179,5,60,5037,...,17416,CA,2019-08-19 15:19:35,3,414,23,2,1,19,1


In [17]:
QuestionnaireListDF = pd.read_sql_table('questionslist', connection)
QuestionnaireListDF

,QuestionNum,Question
0,Q1,"""I would never audition to be on a game show."""
1,Q2,"""I am not much of a flirt."""
2,Q3,"""I have to psych myself up before I am brave e..."
3,Q4,"""I would hate living with room mates."""
4,Q5,"""I mostly listen to people in conversations."""
...,...,...
86,Q87,"""I am not really interested in others."""
87,Q88,"""I make people feel at ease."""
88,Q89,"""I don't mind being the center of attention."""
89,Q90,"""I start conversations."""
